In [1]:
import billboard as bb
import pandas as pd

## Spotify secrets

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
# did this work? commenting out so secrets are safe
# print(os.getenv('SPOTIFY_CLIENT_ID'))
# print(os.getenv('SPOTIFY_CLIENT_SECRET'))

True

In [3]:
spotify_client_id  = os.getenv('SPOTIFY_CLIENT_ID')
spotify_client_secret  = os.getenv('SPOTIFY_CLIENT_SECRET')
genius_client_id = os.getenv('GENIUS_CLIENT_SECRET')
genius_client_secret = os.getenv('GENIUS_CLIENT_SECRET')
genius_access_token = os.getenv('GENIUS_ACCESS_TOKEN')

# Billboard test dataset

In [4]:
# bb.charts(year_end=True)

In [5]:
track = []
artist = []
chart_type = 'billboard-global-200' # weekly charts
date = '2021-03-01' # start date. anything prior to this in 2021 was all Christmas

chart = bb.ChartData(chart_type, date=date)
    

for date in chart:
    for i in range(len(chart)):
        t = chart[i].title
        a = chart[i].artist
        track.append(t)
        artist.append(a)

    chart = bb.ChartData(chart_type, date=chart.nextDate) #lets figure out the erroring out later as this still gives me what i want

AttributeError: 'ChartData' object has no attribute 'nextDate'

In [42]:
f = pd.DataFrame()
f['track'] = pd.Series(track)
f['artist'] = pd.Series(artist)

In [43]:
f = f.drop_duplicates()

track = f.track.tolist()
artist = f.artist.tolist()

## lyricsgenius search_song method
https://pypi.org/project/lyricsgenius/

In [44]:
import lyricsgenius
genius = lyricsgenius.Genius(genius_access_token, skip_non_songs=True, remove_section_headers=True)
genius.verbose = False

In [45]:
#Genius search doesn't utilize an artist featuring another artist, so I want to drop everything after 'Featuring'
sep = 'Featuring'
artist = [x.split(sep,1)[0] for x in artist]
#spotify also don't like artists like H.E.R. but prefers 'her'
artist = [x.replace('.','') for x in artist]

In [46]:
d = {'track':track,'artist':artist}

In [47]:
search = []

for i in range(len(track)):
    hit = track[i]+ ' ' + artist[i]
    search.append(hit)

In [48]:
len(search)

434

In [49]:
genius.search_song(search[0])

Song(id, artist, ...)

In [50]:
genius_lyrics = []
errored_songs = []
tracks = []
artists = []

for i in range(len(search)):
    try:
        test_song = genius.search_song(search[i])
        words = test_song.lyrics
        t = track[i]
        a = artist[i]
    except: 
        errored_songs.append(search[i])
        print('womp womp')
    else:
        genius_lyrics.append(words)
        tracks.append(t)
        artists.append(a)


womp womp
womp womp
womp womp
womp womp
womp womp
womp womp
womp womp
womp womp
womp womp
womp womp
womp womp
womp womp


In [51]:
search[0]

'Drivers License Olivia Rodrigo'

In [52]:
type(genius_lyrics)
len(genius_lyrics)

422

In [53]:
str(genius_lyrics[1])

"Ooh\nNa-na, yeah\n\nI saw you dancing in a crowded room (Uh)\nYou look so happy when I'm not with you\nBut then you saw me, caught you by surprise\nA single teardrop falling from your eye\n\nI don't know why I run away\nI make you cry when I run away\n\nYou could've asked me why I broke your heart\nYou could've told me that you fell apart\nBut you walked past me like I wasn't there\nAnd just pretended like you didn't care\n\nI don't know why I run away\nI make you cry when I run away\n\nTake me back 'cause I wanna stay\nSave your tears for another\nSave your tears for another day\nSave your tears for another day (So)\n\nI made you think that I would always stay\nI said some things that I should never say\nYeah, I broke your heart like someone did to mine\nAnd now you won't love me for a second time\n\nI don't know why I run away, oh, girl\nSaid, I make you cry when I run away\n\nGirl, take me back 'cause I wanna stay\nSave your tears for another\nI realize that I'm much too late\nAnd 

In [54]:
# type(track_artist)
# len(track_artist)

In [55]:
genius_lyrics = pd.Series((genius_lyrics[x] for x in range(len(genius_lyrics))))

track = pd.Series((tracks[x] for x in range(0,len(tracks))))

artist = pd.Series((artists[x] for x in range(0,len(artists))))

In [56]:
# track

In [57]:
# genius_lyrics

In [58]:
data = {'track':track,
        'artist':artist,
        'lyrics':genius_lyrics}
final_df = pd.concat(data, axis = 1)

In [59]:
final_df

,track,artist,lyrics
0,Drivers License,Olivia Rodrigo,I got my driver's license last week\nJust like...
1,Save Your Tears,The Weeknd,"Ooh\nNa-na, yeah\n\nI saw you dancing in a cro..."
2,Blinding Lights,The Weeknd,Yeah\n\nI've been tryna call\nI've been on my ...
3,Calling My Phone,Lil Tjay,Steady callin' my phone\nI done told you befor...
4,Up,Cardi B,"Up, up, up (Ayy), up (Uh), up, look (This is f..."
...,...,...,...
417,Saturday,twenty one pilots,"Slow down on Monday\nNot a sound on Wednesday,..."
418,24 Hours,A Boogie Wit da Hoodie,How you gon' treat me like I'm see-through?\nS...
419,Malibu,Sangiovanni,"Son le parole tue, no?\nTu non bisticci mai?\n..."
420,512,Mora & Jhay Cortez,Hoy dice que llega despué' de la' doce\nDespué...


In [60]:
final_df.to_csv('../genius_lyricsglobal.csv',index=False) #commenting out so i don't accidentally rerun

In [28]:
# with open("../lyrics.txt", "w", encoding='utf-8') as output:
#     output.write(str(genius_lyrics))